In [1]:
import time
import csv
import pickle
import pandas as pd
import numpy as np
import operator
import os

In [2]:
#Only select session which has final purchase
purchase = pd.read_csv('../diginetica/train-purchases.csv', sep=';')[['sessionId', 'itemId']]
print('Total purchase', len(purchase))
queries = pd.read_csv('../diginetica/train-queries.csv', sep=';')[['queryId', 'sessionId', 'items']]
print('Total queries', len(queries))
clicks = pd.read_csv('../diginetica/train-clicks.csv', sep=';')[['queryId', 'itemId']]
print('Total clicks', len(clicks))

Total purchase 18025


/home/xb6cf/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Total queries 923127
Total clicks 1127764


In [3]:
#Session-Purchase Dictionary
class SP_Dictionary(object):
    def __init__(self):
        self.session_dic={}
        self.multiple=False
        self.unique = 0
        
    def add_item(self, session_id, item_purchase):
        if session_id not in self.session_dic:
            self.session_dic[session_id]=[item_purchase]
            self.unique+=1
        else:
            self.session_dic[session_id].append(item_purchase)
            self.multiple=True
    
    def add_item_unique(self, session_id, item_purchase):
        if session_id not in self.session_dic:
            self.session_dic[session_id]=[item_purchase]
            self.unique+=1
        else: #Remove keys with multiple purchases
            self.session_dic.pop(session_id)
            self.unique=self.unique-1
            
    def If_multiple(self):
        if self.multiple:
            print("Number of sessions with only one purchase is:")
            print(self.unique)
        return self.multiple
    
    def get_Dic(self):
        return self.session_dic
    
    def del_key(self,key):
        self.session_dic.pop(key)
    
    def length(self):
        return len(self.session_dic)

In [4]:
#Query-Session Dictionary
class QS_Dictionary(object):
    def __init__(self):
        self.query_dic={}
        
    def add_session(self, query, session_id, sp_dictionary):
        if session_id in sp_dictionary.get_Dic():
            self.query_dic[query]=session_id
            
    def get_Dic(self):
        return self.query_dic
    
    def del_key(self,session_id):
        for key in self.query_dic: 
            if self.query_dic[key] == session_id:
                self.query_dic.pop(key)
    
    def length(self):
        return len(self.query_dic)

In [5]:
#Session-recommendation dictionary
class SR_Dictionary(object):
    def __init__(self):
        self.session_dic={}
        
    def add_recom(self, session_id, recommendation, sp_dictionary):
        if session_id in sp_dictionary.get_Dic():
            if session_id not in self.session_dic:
                self.session_dic[session_id] = [int(recommendation[0])]
                for i in range(len(recommendation)-1):
                    self.session_dic[session_id].append(int(recommendation[i+1]))
            '''
            else:
                for i in range(len(recommendation)):
                    self.session_dic[session_id].append(int(recommendation[i]))
            '''        
    def get_Dic(self):
        return self.session_dic
    
    def del_key(self,key):
        self.session_dic.pop(key)
    
    def length(self):
        return len(self.session_dic)

In [6]:
#Session-click dictionary
class SC_Dictionary(object):
    def __init__(self):
        self.session_dic={}
        
    def add_click(self, query, click, sp_dictionary, qs_dictionary):
        if query in qs_dictionary.get_Dic():
            session_id = qs_dictionary.get_Dic()[query]
            assert session_id in sp_dictionary.get_Dic()
            if session_id not in self.session_dic:
                self.session_dic[session_id] = [click]
            else:
                self.session_dic[session_id].append(click)
                
    def get_Dic(self):
        return self.session_dic
    
    def del_key(self,key):
        self.session_dic.pop(key)
    
    def length(self):
        return len(self.session_dic)

In [7]:
def del_session(session_id, session_purchase, session_recom, query_session):
    session_purchase.del_key(session_id)
    session_recom.del_key(session_id)
    #query_session.del_key(session_id)

In [8]:
session_purchase = SP_Dictionary()
query_session = QS_Dictionary()
session_click = SC_Dictionary()
session_recom = SR_Dictionary()

for sp in purchase.values:
    #session_purchase.add_item(sp[0], sp[1])
    session_purchase.add_item_unique(sp[0], sp[1])
if session_purchase.If_multiple()==True:
    print("One session has multiple purchases!")
else:
    print("Sessions only have unique purchases!")
for qsr in queries.values:
    query_session.add_session(qsr[0], qsr[1], session_purchase)
    recom = qsr[2].split(",")
    session_recom.add_recom(qsr[1], recom, session_purchase)
for qc in clicks.values:
    session_click.add_click(qc[0], qc[1], session_purchase, query_session)
    
#Filter sessions by clicks
filter_key_other = []
filter_key_click = []
#The session only has purchase, no clicks
for key in session_purchase.get_Dic():
    if key not in session_click.get_Dic():
        filter_key_other.append(key)
#The session has less than 2 clicks        
for key in session_click.get_Dic():
    if len(session_click.get_Dic()[key]) < 2:
        filter_key_click.append(key)
        filter_key_other.append(key)
for key in filter_key_click:
    session_click.del_key(key)
for key in filter_key_other:
    del_session(key, session_purchase, session_recom, query_session)
assert session_click.length()==session_recom.length() and session_click.length()==session_purchase.length()

Sessions only have unique purchases!


In [28]:
##Extract session recommend and clicks; relation between views and clicks?
class Session(object):
    """Instance that represent a sample of train/dev/test corpus."""

    def __init__(self, session_id):
        self.session_id = session_id
        self.purchase = None
        self.recommend = None
        self.clicks = None
        
    def Add_recommend(self, session_id, session_recom):
        assert session_id == self.session_id
        self.recommend=session_recom[session_id]            
                
    def Add_click(self, session_id, session_click):
        assert session_id == self.session_id
        self.clicks=session_click[session_id]
            
    def Add_purchase(self, session_id, session_purchase):
        assert session_id == self.session_id
        self.purchase=session_purchase[session_id]
        
    def write_session_clicks(self, file):
        for cli in self.clicks:
            file.write(str(cli) + ' ')
        file.write(str(self.purchase[0]) + '\n')

In [29]:
def Sample(session_purchase, session_click, session_recom):
    session_list = []
    savefile=open(os.path.join('../','dig_data.txt'),'a+')
    for session_id in session_purchase.get_Dic():
        sess = Session(session_id)
        sess.Add_recommend(session_id, session_recom.get_Dic())
        sess.Add_click(session_id, session_click.get_Dic())
        sess.Add_purchase(session_id, session_purchase.get_Dic())
        sess.write_session_clicks(savefile)
        session_list.append(sess)
    return session_list

In [30]:
session_list = Sample(session_purchase, session_click, session_recom)

In [20]:
session_click.get_Dic()[570]

[4523, 4523]

In [21]:
session_purchase.get_Dic()[570]

[4523]